## Extracción de datos automática

### Acceso al sitio

In [0]:
%pip install beautifulsoup4

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#Importar librerías.
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime
import pickle

In [0]:
#URL de Fuente: TLC Trip Record Data
tlc_url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#Solicitar URL y obtener objeto de respuesta.
tlc_respuesta = requests.get(tlc_url)

#Analizar el texto obtenido.
tlc_sopa = BeautifulSoup(tlc_respuesta.text, 'html.parser')

#Encontrar todos los hipervínculos presentes en la página web.
tlc_vinculos = tlc_sopa.find_all('a')

In [0]:
#Leer diccionarios de referencia.
directorio_diccionarios = "/dbfs/d_amarillo.pickle"
d_amarillo = pd.read_pickle(directorio_diccionarios)

### Descarga de nuevos archivos

Acceso a Blob Storage

In [0]:
#Datos de acceso
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="
container = "yellowtaxis"

In [0]:
#Variable de configuración Spark
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

Descarga

In [0]:
#Permitir a Apache Arrow convertir Pandas a pySpark DF:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
#Obtener el año en curso.
date = datetime.date.today()
año = '{0}'.format(date.year)

In [0]:
#Comparar con diccionario actual. Agregar (y exportar) sólo nueva variable.
for vinculo in tlc_vinculos:
    if(
        'yellow' in vinculo.get('href', []) and
        '.parquet' in vinculo.get('href', [])
    ):
        if (
            año in vinculo.get('href', [])
        ):
            hipervinculo = vinculo.get('href', [])
            mes = hipervinculo[69]+hipervinculo[70]

            #Leer el vínculo y compararlo con las otras entradas del diccionario.

            for clave, valor in d_amarillo.items():
                valor_año = valor[7]+valor[8]+valor[9]+valor[10]
                valor_mes = valor[13]+valor[14]
                
                if (
                    (año == valor_año) and
                    (mes == valor_mes)
                ):
                    equal = True
                    break
                else:
                    equal = False

            if (equal == False):
                #Agregar entrada nueva al diccionario.
                d_amarillo["yellow_{0}_{1}".format(año, mes)] = "yellow_{0}_{1} = pd.read_parquet('{2}')".format(año, mes, hipervinculo)

                #Exportar diccionario actualizado a almacenamiento conveniente.

                amarillo = open(directorio_diccionarios, 'wb')
                pickle.dump(d_amarillo, amarillo)
                amarillo.close()

                #Exportar nuevo archivo parquet (tabla) a almacenamiento previo (raw).

                #Ejecutar el comando
                exec(d_amarillo["yellow_{0}_{1}".format(año, mes)])

                #Pasar dataframe a parquet en la ubicación

                dataframe = spark.createDataFrame(vars()[list(d_amarillo)[-1]])

                dataframe.write.format("parquet").save(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/yellow_{año}_{mes}.parquet")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3007146002323698> in <cell line: 2>()
     45                 dataframe = spark.createDataFrame(vars()[list(d_amarillo)[-1]])
     46 
---> 47                 dataframe.write.format("parquet").save(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/yellow_{año}_{mes}.parquet")

NameError: name 'container' is not defined